# Modeling
In this notebook I will use the Ames housing data which was prepared in the previous notebooks to predict `SalePrice`.

## Load Data

In [2]:
import joblib

# Load split and scaled data
X_train = joblib.load("../data/X_train_scaled.pkl")
X_test = joblib.load("../data/X_test_scaled.pkl")
y_train = joblib.load("../data/y_train.pkl")
y_test = joblib.load("../data/y_test.pkl")